<a href="https://colab.research.google.com/github/eksq1/dodo/blob/main/inclass/mobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MainActivity

In [1]:
package com.android.dmu2401finalexam

import android.content.Intent
import androidx.appcompat.app.AppCompatActivity
import android.os.Bundle
import android.os.Environment
import android.text.Editable
import android.text.TextWatcher
import android.util.Log
import com.android.dmu2401finalexam.databinding.ActivityMainBinding
import java.io.BufferedReader
import java.io.File
import java.io.FileOutputStream
import java.io.FileReader

class MainActivity : AppCompatActivity() {

    // binding 변수의 데이터 타입은 레이아웃 파일의 파일 이름으로부터 유추 가능
    lateinit var binding: ActivityMainBinding

    // SharedPreference 영역의 파일 이름을 가져오면 코드가 길어지기 때문에
    // 파일 이름을 관리할 임시 변수 선언
    lateinit var tmpFilename: String

    var isSaved: Boolean = false   // 공유 데이터로, 메모 내용이 저장되었는지 여부를 관리
                                    // false: 저장안됨, true: 저장됨
    var state: Int = 1              // 초기 시작 상태를 구분하는 상태 코드 선언/정의

    // Activity 이동을 위한 intent 객체 선언/정의
    lateinit var filenameIntent: Intent
    lateinit var filelistIntent: Intent

    // 파일 저장 위치 설정
    val fileSaveLocation = Environment.getExternalStoragePublicDirectory(Environment.DIRECTORY_DOCUMENTS).toString()

    // lifecycle - 첫 번째 함수 onCreate 시작
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        //setContentView(R.layout.activity_main)
        // binding 변수 초기화
        binding = ActivityMainBinding.inflate(layoutInflater)
        setContentView(binding.root)

        filenameIntent =  Intent(this, FilenameActivity::class.java)
        filelistIntent =  Intent(this, FilelistActivity::class.java)

        // 공유 데이터 초기화 - 마지막 값이 android에 저장되기 때문에
        MyApplication.preferences.setString("filename", "new")
        MyApplication.preferences.setBoolean("isSaved", false)
        MyApplication.preferences.setString("saveLocation", fileSaveLocation)

        // 공유 데이터에서 파일 이름과 메모 저장 상태 가져오기
        tmpFilename = MyApplication.preferences.getString("filename", "new")
        isSaved = MyApplication.preferences.getBoolean("isSaved", false)
        Log.d("myCheck", "시작할 때 파일 이름은 ${tmpFilename}")
        Log.d("myCheck", "시작할 때 저장 상태는 ${isSaved}")

        // filenameToSave의 값을 "파일 이름"을 나타내는 컴포넌트에 표시
        binding.tvFilename.setText(tmpFilename)
        // 메모 내용이 저장된 상태이면 "메모 저장"을 비활성화하는 것으로
        // 저장해야할 필요가 있으면, 즉 isSaved == false이면 활성화한다.
        if(! isSaved) {
            binding.btnSave.isEnabled = false
        } else {  // 초기 상태이기 때문에 여기서 else 이하는 필요하지 않음
            binding.btnSave.isEnabled = true
        }

        // 초기 시작 상태를 구분하는 상태 코드 정의
        state = 1

        binding.btnNewMemo.setOnClickListener() {
            Log.d("myCheck", "새 메모 버튼 클릭")
            state = 2
            if(isSaved || binding.edMemo.text.toString().length == 0 ){
                // 저장 되었거나, 메모 내용의 길이가 0인 경우 다시 저장할 필요가 없어
                // 메모 내용 clear하고, 파일 이름을 초기화 등
                Log.d("myCheck", "이미 저장되었거나 메모 내용이 없는 경우 ")
                binding.edMemo.setText("")
                MyApplication.preferences.setString("filename", "new")
                tmpFilename = "new"
                binding.tvFilename.setText(tmpFilename)
                MyApplication.preferences.setBoolean("isSaved", false)
                isSaved = false
                binding.btnSave.isEnabled = false
            } else {
                // 저장되지 않았거나 메모 내용이 있을 때 저장해야 한다.
                // 파일 이름이 있는 경우와 없는 경우 구분
                if(tmpFilename == "new" || tmpFilename == "new*") {
                    // 파일 이름이 없어 파일을 입력받기 위한 Activity 이동 후 파일 이름을 입력 받아 돌아온 뒤에
                    // onResume에서 state 구분해서 2인 경우 받아온 파일 이름으로 저장
                    Log.d("myCheck", "현재 파일 이름이 new인 경우 ")
                    startActivity( filenameIntent )  // 파일 이름을 입력받기 위한 intent 이동
                } else {
                    // 파일 이름이 있는 경우
                    // 파일 저장
                    Log.d("myCheck", "이미 파일 이름이 존재하는 경우, 저장하고 clear")
                    saveMemo()
                    binding.edMemo.setText("")
                    MyApplication.preferences.setString("filename", "new")
                    tmpFilename = "new"
                    binding.tvFilename.setText(tmpFilename)
                    MyApplication.preferences.setBoolean("isSaved", false)
                    isSaved = false
                    binding.btnSave.isEnabled = false
                }
            }
        }

        binding.btnSave.setOnClickListener() {
            Log.d("myCheck", "메모 저장 버튼을 클릭한 경우 ")
            state = 4  // 메모 저장 버튼을 클릭해서 intent 이동후 복귀할 때 구분하기 위한 값
            if (tmpFilename == "new" || tmpFilename == "new*") {
                Log.d("myCheck", "현재 파일 이름이 new인 경우 ")
                // 파일 이름이 없어 파일을 입력받기 위한 Activity 이동 후 파일 이름을 입력 받아 돌아온 뒤에
                // onResume에서 state 구분해서 4인 경우 받아온 파일 이름으로 저장
                startActivity(filenameIntent)  // 파일 이름을 입력받기 위한 intent 이동            }
                // 파일 이름을 입력받고 온 경우는 onResume에서 처리해야 함.
            } else {
                // 파일 이름이 있는 경우
                // 파일 저장
                Log.d("myCheck", "현재 파일 이름이 있는 경우 저장하고, 파일 이름 뒤에 * 제거")
                saveMemo()
                // 화면에 파일 이름 뒤에 표시된 "*"를 제거해야 한다.
                if( binding.tvFilename.text.toString().substring(binding.tvFilename.text.toString().length - 1 ) == "*") {
                    binding.tvFilename.setText(
                        // 0번째 문자부터 (길이 - 1)까지만 다시 화며에 표시한다.
                        binding.tvFilename.text.toString().substring(0, binding.tvFilename.text.toString().length - 1)
                    )
                }
            }
        }

        binding.btnOpen.setOnClickListener() {
            Log.d("myCheck", "메모 열기 버튼을 클릭한 경우 ")
            Log.d("myCheck", "파일 열기에서 isSaved는 ${isSaved}")
            Log.d("myCheck", "파일 열기에서 메모 내용의 길이는 ${binding.edMemo.text.toString()?.length?:0}")
            if(isSaved || binding.edMemo.text.toString().length == 0 ){
                // 현재 메모를 저장할 필요가 없는 경우
                state = 8
                // 파일 목록 activity로 이동
                startActivity(filelistIntent)
            } else {
                // 저장되지 않았거나 메모 내용이 있을 때 저장해야 한다.
                // 파일 이름이 있는 경우와 없는 경우 구분
                // 현재 메모를 저장해야 하는 경우
                if(tmpFilename == "new" || tmpFilename == "new*") {
                    state = 12
                    // 파일 이름이 없어 파일을 입력받기 위한 Activity 이동 후 파일 이름을 입력 받아 돌아온 뒤에
                    // onResume에서 state 구분해서 2인 경우 받아온 파일 이름으로 저장
                    Log.d("myCheck", "현재 파일 이름이 new인 경우 ")
                    // 파일 이름이 없기 때문에 먼저 파일 이름 입력을 위한 인텐트 이동 필요
                    // onResume에서 state == 12인 경우, 다시 파일 목록을 나타내는 intent로 이동
                    // onResume에서 state = 8로 다시 설정해야 함.
                    startActivity( filenameIntent )  // 파일 이름을 입력받기 위한 intent 이동
                } else {
                    state = 8
                    // 파일 이름이 있는 경우
                    // 파일 저장
                    Log.d("myCheck", "이미 파일 이름이 존재하는 경우, 저장하고 clear")
                    saveMemo()
                    // 파일 목록 activity로 이동
                    startActivity(filelistIntent)
                }
            }
        }

        // editText 메모 컴포넌트에서 데이터 변화가 있는지 이벤트 처리
        binding.edMemo.addTextChangedListener( object: TextWatcher {
            // TextWatcher는 기본 3가지 함수를 override해야 한다.
            override fun beforeTextChanged(s: CharSequence?, start: Int, count: Int, after: Int) {
                // editText 컴포넌트에서 글자를 입력하기 직전
            }

            override fun onTextChanged(s: CharSequence?, start: Int, before: Int, count: Int) {
                // editText 컴포넌트에서 글자를 입력할 때
                // 글자 입력이 있었기에 저장해야 하므로, 공유 데이터에서 해당 변수값을 true로 설정
                MyApplication.preferences.setBoolean("isSaved", true)
                // 저장해야 하므로, 화면 UI에서 "메모 저장" 버튼 활성화
                binding.btnSave.isEnabled = true
                // 현재 표시되고 있는 파일 이름의 끝에 한 글자 띄어내기 - '*'이면 '저장 전', 아니면 '저장 후'
                var tmp = binding.tvFilename.text.toString().substring(
                    binding.tvFilename.text.toString().length - 1)
                // '*'이 아니면, "*" 붙이기 - 수정되었다는 표시
                if(tmp != "*") {
                    // 파일 이름 뒤에 "*" 붙이는 것은 화면에서만 보이도록 함.
                    // 공유 데이터 영역에 저장된 "filename"의 값에는 붙이지 않는다.
                    binding.tvFilename.text = binding.tvFilename.text.toString() + "*"
                }
            }

            override fun afterTextChanged(s: Editable?) {
                // editText 컴포넌트에서 글자를 입력한 뒤에
                // onTextChanged에 있는 내용을 afterTextChanged에서 해도 관계 없음.
            }
        })
    } // onCreate의 끝

    // 파일 저장 함수
    fun saveMemo() {
        tmpFilename = MyApplication.preferences.getString("filename", "")
        Log.d("myCheck", "메모 내용을 [${tmpFilename}](으)로 저장.")
        val baseDir = MyApplication.preferences.getString("saveLocation", ".")
        val file = File(baseDir, tmpFilename)
        val fileOutputStream = FileOutputStream(file)
        fileOutputStream.write(binding.edMemo.text.toString().toByteArray())
        fileOutputStream.close()
    }

    // 파일 열기 함수
    fun openMemo() {
        tmpFilename = MyApplication.preferences.getString("filename", "")
        Log.d("myCheck", "선택한 [${tmpFilename}] 메모 파일 열기")
        val baseDir = MyApplication.preferences.getString("saveLocation", ".")
        val file = baseDir + "/" + tmpFilename
        val reader = FileReader(file)
        var buffer = BufferedReader(reader)

        var readContent: String = ""
        var temp: String?
        while(true) {
            temp = buffer.readLine()
            if(temp == null) break
            else readContent += temp + "\n"
        }

        binding.edMemo.setText(readContent)
        binding.tvFilename.setText(tmpFilename)
    }


    override fun onResume() {
        super.onResume()
        // onResume 함수로 진입하는 경우를 구분해야 함.
        // 아래 when 블록에서 state 값에 따라 중복되는 코드가 있음, 별도의 함수로 분리해야 하나,
        // 이해를 쉽게하기 위해 중복된 코드를 그대로 사용함.
        when(state) {
            1 -> {
                    // 초기 상태에서 onResume으로 진입한 경우
                    // 이 경우는 할 일이 없음
            }
            2 -> {
                    // 새 메모 버튼을 클릭했을 때 파일 이름 입력 받기위한 Intent 이동후 onResume으로 진입한 경우
                    // 작성 중인 메모가 저장되지 않았기 때문에, 파일 이름을 입력받은 것이다.
                    // 입력된 파일 이름으로 메모 저장하고,
                    // 현재 메모의 내용을 화면에서 지우고, 파일 이름 초기화하고, 메모 저장 버튼의 상태도 초기화한다.
                    // 입력된 파일 이름 가져오기
                    tmpFilename = MyApplication.preferences.getString("filename", "")
                    // 메모 저장
                    saveMemo()
                    // 초기화
                    binding.edMemo.setText("")
                    MyApplication.preferences.setString("filename", "new")
                    tmpFilename = "new"
                    binding.tvFilename.setText(tmpFilename)
                    MyApplication.preferences.setBoolean("isSaved", false)
                    isSaved = false
                    binding.btnSave.isEnabled = false
            }
            4 -> {
                    // 메모 저장 버튼을 클릭했을 때 파일 이름 입력 받기위한 Intent 이동후 onResume으로 진입한 경우
                    // state == 2일 때와 같지만, 초기화가 없다 .
                    // 입력된 파일 이름 가져오기
                    tmpFilename = MyApplication.preferences.getString("filename", "")
                    binding.tvFilename.setText(tmpFilename)
                    // 메모 저장
                    saveMemo()
            }
            8 -> {
                    // 메모 열기 버튼을 클릭했을 때 파일 이름 입력 받기위한 Intent 이동후 onResume으로 진입한 경우
                    // 파일 목록으로 이동하기 전에 현재 메모 내용이 한 번도 저장되지 않은 경우로
                    // state == 2 또는 state == 4인 경우와 같지만, 초기화가 없고
                    // 가져온 파일 목록으로 파일을 열어야 한다.
                    // 입력된 파일 이름 가져오기
                    //tmpFilename = MyApplication.preferences.getString("filename", "")
                    // 메모 열기
                    openMemo()
            }
            12 -> {
                // 메모 열기에서 파일 이름이 없어 먼저 filenameIntent로 이동했다가 돌아온 경우
                // 파일 저장
                // 파일 목록 intent로 이동
                state = 8
                startActivity(filelistIntent)
                // 파일 리스트 선택하고 다시 state == 8로 진행...
            }
        }
    }

    override fun onDestroy() {
        super.onDestroy()
        Log.d("myCheck", "기말 시험 보느라 수고하셨습니다.")
    }
}

SyntaxError: invalid decimal literal (<ipython-input-1-0323f0968ff4>, line 75)

# PreferenceUtil

In [3]:
package com.android.dmu2401finalexam

import android.content.Context
import android.content.SharedPreferences

class PreferenceUtil(context: Context) {

    private val preferences: SharedPreferences =
        context.getSharedPreferences("shared_data", Context.MODE_PRIVATE)

    fun getString(key: String, shared_data: String): String {
        return preferences.getString(key, shared_data).toString()
    }

    fun getBoolean(key: String, shared_data: Boolean): Boolean {
        return preferences.getBoolean(key, shared_data)
    }

    fun setString(key: String, shared_data: String) {
        preferences.edit().putString(key, shared_data).apply()
    }

    fun setBoolean(key: String, shared_data: Boolean) {
        preferences.edit().putBoolean(key, shared_data).apply()
    }
}

SyntaxError: invalid syntax (<ipython-input-3-8b461be4e7c2>, line 1)

# MyApplication

In [4]:
package com.android.dmu2401finalexam

import android.app.Application

class #: Application() {

    companion object {
        lateinit var preferences: PreferenceUtil     // 사용자 정의 타입(클래스)
    }

    override fun onCreate() {
        super.onCreate()
        preferences = PreferenceUtil(applicationContext)

    }
}

SyntaxError: unmatched '}' (<ipython-input-4-5b2afa283c8b>, line 16)

# FilenameActivity

In [5]:
package com.android.dmu2401finalexam

import androidx.appcompat.app.AppCompatActivity
import android.os.Bundle
import android.widget.Toast
import com.android.dmu2401finalexam.databinding.ActivityFilenameBinding

class FilenameActivity : AppCompatActivity() {
    // viewBinding 사용을 위한 binding 변수 선언
    lateinit var binding: ActivityFilenameBinding
    var filename: String? = ""

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        //setContentView(R.layout.activity_filename)
        // viewBinding 사용을 위한 binding 변수 초기화
        binding = ActivityFilenameBinding.inflate(layoutInflater)
        setContentView(binding.root)

        // 파일이름을 입력하는 레이아웃에서 "확인" 버튼 이벤트 처리
        binding.btnConfirm.setOnClickListener() {
            // 입력된 파일 이름 검증
            // 길이가 0이 아이어야 한다.
            // 확장자는 자동으로 .txt를 붙인다.
            filename = binding.edFilename.text.toString()
            if(filename!!.length >= 1) {
                if (!filename!!.contains(".")) {
                    // 파일 이름의 길이가 1이상이고, 이름에 "."을 포힘하지 않는 경우
                    // shared preference에 키를 "filename"으로하고, filename 변수의 값을 저장한다.
                    MyApplication.preferences.setString("filename", filename!! + ".txt")
                    // 파일 이름이 결정되었기에, 파일 이름 입력을 위한 Intent 종료
                    finish()
                } else {
                    // 파일 이름에 확장자가 포함된 경우 Toast 메시지를 나타낸다.
                    Toast.makeText(this, "파일 이름에 확장자가 포함되어 있습니다.", Toast.LENGTH_SHORT).show()
                }
            } else {
                // 파일 이름의 길이가 1보다 작은 경우로 파일 이름이 입력되지 않은 경우
                // Toast 메시지를 나타낸다.
                Toast.makeText(this, "파일 이름이 입력되지 않았습니다.", Toast.LENGTH_SHORT).show()
            }
        }
    }
}

SyntaxError: invalid decimal literal (<ipython-input-5-ca129e67ff55>, line 23)

# FilelistActivity

In [6]:
package com.android.dmu2401finalexam

import android.R
import androidx.appcompat.app.AppCompatActivity
import android.os.Bundle
import android.os.Environment
import android.util.Log
import android.widget.ArrayAdapter
import com.android.dmu2401finalexam.databinding.ActivityFilelistBinding
import java.io.File

class FilelistActivity : AppCompatActivity() {
    // viewBinding 사용을 위한 binding 변수 선언
    lateinit var binding: ActivityFilelistBinding
    lateinit var filelistArray: ArrayList<String>

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        //setContentView(R.layout.activity_filelist)
        // viewBinding 사용을 위한 binding 변수 초기화
        binding = ActivityFilelistBinding.inflate(layoutInflater)
        setContentView(binding.root)

        Log.d("myCheck", "FilelistAcctivity 시작")
        // 지정된 디렉터리에서 목록을 읽어
        // 파일 이름만 arrayList에 담아
        // arrayList를 이용해서 adapter 생성
        // adapter를 listview 컴포넌트의 adapter 속성에 할당
        // 지정된 디렉터리 위치는 ExternalStoragePublicDirectory에서 "문서(DIRECTORY_Documents)" 디렉터리로 한다.
        // 디렉터리 위치를 문자열로 할당
        var rootDirectory = MyApplication.preferences.getString("saveLocation", ".")
        Log.d("myCheck", "FilelistAcctivity - ${rootDirectory}")
        // 문자열 값인 rootDirectory를 사용해서 File 객체 생성
        var fileDir = File(rootDirectory)
        // fileDir에서 파일 목록을 읽어 fileArrayList에 할당
        var filelist = fileDir.listFiles()
        filelistArray = ArrayList<String>()

        Log.d("myCheck", "FilelistAcctivity - ${filelist}")
        // filelist에서 파일의 이름을 하나씩 읽어 배열에 추가하여 파일 이름을 담는 문자열 배열을 생성한다.
        for(i in 0 until filelist.size) {
            Log.d("myCheck<", "${filelist[i]} 번째 파일 목록 추가 ")
            Log.d("myCheck<", "파일 구분 : ${filelist[i].name}  ")
            if(!filelist[i].isDirectory) {
                // 지정된 이름 항목이 디렉터리가 아닌 경우
                filelistArray.add(filelist[i].name)           // filelist[i] 번째 항목의 이름을 배열에 추가
                Log.d("myCheck", "FilelistAcctivity - 파일 목록 : ${filelist[i]}")
            }
        }

        // 문자열 배열을 이용하여 adapter를 만들고, listView의 adapter 속성에 할당한다.
        binding.listView.adapter = ArrayAdapter<String>(this, R.layout.simple_list_item_1, filelistArray)

        // listView에서 하나의 항목을 클릭할 때 이벤트 처리
        binding.listView.setOnItemClickListener() { adapterView, view, i, l ->
            // 3번째 인수인 i가 listView에서 선택된 항목의 인덱스이다.
            // 파일을 열기 위한 공유 변수에 저장
            MyApplication.preferences.setString("filename", filelistArray[i])
            Log.d("myCheck", "FilelistActivity 종료 - finish 직전")
            finish()    // 파일 목록을 나타내는 intent 종료하고 main으로 돌아간다.
        }
    }
}

SyntaxError: invalid decimal literal (<ipython-input-6-89f40658c151>, line 56)

# box_boreder.xml

In [2]:
<?xml version="1.0" encoding="utf-8"?>
<shape xmlns:android="http://schemas.android.com/apk/res/android"
    android:shape="rectangle">
    <stroke android:width="2dp" android:color="#ff0000"/>
    <corners android:radius="5dp" />
    <solid android:color="#ffffff" />
</shape>

SyntaxError: invalid syntax (<ipython-input-2-aae328a408bd>, line 1)